# Demonstration of downsampling biosamples to only unique cellular conditions
Based on 733 sample x 3.5+ million DHS presence/absence matrix

# 1. Import libraries & load datasets

In [1]:
from platform import python_version
print(python_version())

3.6.4


In [2]:
import sys
import numpy as np
import pandas as pd
import gzip
import matplotlib.pyplot as plt
from sklearn.decomposition import NMF, non_negative_factorization

In [3]:
sys.path.append('..')
import OONMFhelpers
import OONMF

#### Fetch the 733-biosample presence/absence Index
This will take at least a few minutes.
These data can be obtained from here: https://doi.org/10.5281/zenodo.3752359

In [4]:
A = pd.read_table('../data/dat_bin_FDR01_hg38.txt.gz', header=None).T
A.shape

(733, 3591898)

#### Load in biosample metadata

In [24]:
metadata = pd.read_csv('../data/replicate_metadata_733biosamples.tsv', sep='\t')
#metadata.columns
metadata.shape

(733, 5)

#### Show distribution of biosamples across "organ systems"

In [27]:
pd.crosstab(index=metadata["system"], columns="count")

col_0,count
system,
Cardiovascular,54
Connective,91
Digestive,60
Embryonic,26
Endocrine,9
Epithelial,23
Fetal Life Support,21
Genitourinary,14
Hematopoietic,109


#### Obtain subset that represents the unique cellular conditions (439 out of 733 biosamples)

In [19]:
unique_cut = metadata['Unique.cellular.condition'].values.astype(bool)
sum(unique_cut)

439

#### Show distribution of biosamples across "organ systems", conditioned on being part of the subset of unique cellular conditions

In [26]:
pd.crosstab(index=metadata["system"], columns=metadata["Unique.cellular.condition"])

Unique.cellular.condition,0,1
system,,
Cardiovascular,16,38
Connective,46,45
Digestive,18,42
Embryonic,13,13
Endocrine,2,7
Epithelial,12,11
Fetal Life Support,4,17
Genitourinary,7,7
Hematopoietic,65,44


#### Subset full dataset based on the subset of unique cellular conditions

In [20]:
A_uniq = A[unique_cut]
A_uniq.shape

(439, 3591898)

# 2. Perform the decomposition using NMF

#### Number of desired components (k) and a random seed

In [21]:
Nc = 16
seed = 20 # (not very important for NNDSVD)

#### Perform NMF with NNDSVD. Requires lots of memory and quite a bit of time
(apologies for the lack of quantification here, YMMV)

In [22]:
a = OONMF.NMFobject(theNcomps=Nc)
a.performNMF(data=A_uniq, randomseed=seed, theinit='nndsvd')

starting NMF at  20200415_22:48:52
done with NMF at  20200415_22:57:17
returning reconstruction error


4580.908428435375

#### Write the output to disk

In [23]:
a.writeNMF(Basis_foutname= '../data/2020-04-15NC16_NNDSVD_uniqOnly_Basis.npy', Mixture_foutname='../data/2020-04-15NC16_NNDSVD_uniqOnly_Mixture.npy')

# 3. Compare result with full dataset
Compare obtained DHS majority components to those obtained using the full 733 biosample dataset

In [41]:
original_decomp = OONMF.NMFobject(16)
original_decomp.matrix_input_name('../data/2018-06-08NC16_NNDSVD_Basis.npy', '../data/2018-06-08NC16_NNDSVD_Mixture.npy')
original_decomp.read_matrix_input(compressed=True)

In [42]:
#majcomp = np.argmax(original_decomp.Basis, axis=1)
majcomp = np.argmax(original_decomp.Mixture, axis=0)
len(majcomp)

3591898

In [43]:
#majcomp_uniq = np.argmax(a.Basis, axis=1)
majcomp_uniq = np.argmax(a.Mixture, axis=0)
len(majcomp_uniq)

3591898

In [ ]:
pd.crosstab(majcomp, columns="count", margins=True)

In [49]:
pd.crosstab(majcomp_uniq, columns="count")

col_0,count
row_0,
0,449158
1,279380
2,314354
3,113517
4,287664
5,394968
6,338905
7,224242
8,103401


In [60]:
pd.crosstab(majcomp, majcomp_uniq, margins=True)

col_0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,All
row_0,,,,,,,,,,,,,,,,,
0,137534,123,773,1250,464,2303,3355,340,151,130,602,345,75,6135,956,3134,157670
1,6521,9370,221,773,668,972,299,751,33,1091,1102,226,77,144,61,33877,56186
2,37062,450,268619,3309,4049,11931,2174,1178,432,9112,735,1082,369,267008,18378,653,626541
3,21488,254489,7945,3593,17259,14949,9690,35012,1955,10772,4597,2700,6687,5297,2496,5954,404883
4,39075,647,1556,347,1752,221683,2525,1683,259,935,336,972,524,6232,1181,485,280192
5,20359,9826,4027,8332,6386,4284,3299,67544,884,1192,902,3622,2133,3369,1822,6106,144087
6,4171,42,422,36,241706,338,426,302,7,63,42,55,19,17110,207,34,264980
7,30063,470,6115,661,1034,1245,299049,556,266,2589,466,573,338,10190,106942,921,461478
8,3772,215,1527,658,1623,1034,3582,640,97543,2342,1235,744,604,1933,1353,64,118869


In [57]:
pd.crosstab(metadata["system"], columns=np.argmax(original_decomp.Basis, axis=1), margins=True)

col_0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,All
system,,,,,,,,,,,,,,,,,
Cardiovascular,3,4,3,1,0,0,0,0,26,0,0,0,0,17,0,0,54
Connective,0,64,0,25,0,0,0,0,0,0,0,0,0,0,0,2,91
Digestive,14,4,0,0,0,0,0,0,0,0,0,2,34,0,0,6,60
Embryonic,1,0,23,0,0,0,0,2,0,0,0,0,0,0,0,0,26
Endocrine,7,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,9
Epithelial,0,7,0,3,0,0,0,0,0,0,0,0,0,0,0,13,23
Fetal Life Support,0,0,0,0,0,0,18,0,1,1,0,0,0,1,0,0,21
Genitourinary,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,14
Hematopoietic,4,0,2,0,57,0,0,0,0,0,0,0,0,0,46,0,109


In [58]:
pd.crosstab(metadata["system"][unique_cut], columns=np.argmax(a.Basis, axis=1), margins=True)

col_0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,All
system,,,,,,,,,,,,,,,,,
Cardiovascular,2,1,1,0,0,0,0,0,22,0,0,0,10,0,0,2,38
Connective,0,20,0,0,0,0,0,0,0,0,0,0,0,0,0,25,45
Digestive,11,1,0,0,0,0,0,2,0,2,0,24,0,0,0,2,42
Embryonic,0,0,10,0,0,0,0,0,0,0,0,0,0,2,1,0,13
Endocrine,5,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,7
Epithelial,0,3,0,0,0,0,0,4,0,0,0,0,0,0,0,4,11
Fetal Life Support,0,0,0,0,12,0,0,0,1,0,0,0,1,3,0,0,17
Genitourinary,3,0,0,0,0,0,0,2,0,0,0,1,0,0,0,1,7
Hematopoietic,4,0,1,0,0,38,0,0,0,0,0,0,0,1,0,0,44


In [61]:
tab_biosamples = pd.crosstab(np.argmax(original_decomp.Basis[unique_cut], axis=1), np.argmax(a.Basis, axis=1), margins=True)
tab_biosamples

col_0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,All
row_0,,,,,,,,,,,,,,,,,
0,52,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,54
1,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,37,43
2,0,0,13,0,0,0,0,0,1,0,0,0,0,4,1,0,19
3,0,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25
4,3,0,0,0,0,25,0,0,0,0,0,0,0,0,0,0,28
5,0,1,0,0,0,0,0,9,0,0,0,0,0,0,0,7,17
6,0,0,0,0,12,0,0,0,0,0,0,0,0,2,0,0,14
7,1,0,0,0,0,0,20,0,0,0,0,0,1,0,14,0,36
8,0,0,0,0,0,0,0,0,22,0,0,0,0,0,0,0,22


In [68]:
round(tab_biosamples.div(tab_biosamples["All"], axis=0)*100).astype(int)

col_0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,All
row_0,,,,,,,,,,,,,,,,,
0,96,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0,100
1,0,14,0,0,0,0,0,0,0,0,0,0,0,0,0,86,100
2,0,0,68,0,0,0,0,0,5,0,0,0,0,21,5,0,100
3,0,100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,100
4,11,0,0,0,0,89,0,0,0,0,0,0,0,0,0,0,100
5,0,6,0,0,0,0,0,53,0,0,0,0,0,0,0,41,100
6,0,0,0,0,86,0,0,0,0,0,0,0,0,14,0,0,100
7,3,0,0,0,0,0,56,0,0,0,0,0,3,0,39,0,100
8,0,0,0,0,0,0,0,0,100,0,0,0,0,0,0,0,100


In [75]:
round(tab_biosamples.T.div(tab_biosamples.T["All"], axis=0)*100).astype(int).T

col_0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,All
row_0,,,,,,,,,,,,,,,,,
0,84,0,0,0,0,0,0,0,0,0,0,0,0,8,6,0,12
1,0,18,0,0,0,0,0,0,0,0,0,0,0,0,0,70,10
2,0,0,93,0,0,0,0,0,4,0,0,0,0,33,6,0,4
3,0,76,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6
4,5,0,0,0,0,66,0,0,0,0,0,0,0,0,0,0,6
5,0,3,0,0,0,0,0,38,0,0,0,0,0,0,0,13,4
6,0,0,0,0,100,0,0,0,0,0,0,0,0,17,0,0,3
7,2,0,0,0,0,0,100,0,0,0,0,0,8,0,78,0,8
8,0,0,0,0,0,0,0,0,96,0,0,0,0,0,0,0,5


In [69]:
pd.crosstab(np.argmax(original_decomp.Basis, axis=1), columns=metadata["Unique.cellular.condition"])

Unique.cellular.condition,0,1
row_0,,
0,36,54
1,64,43
2,15,19
3,8,25
4,29,28
5,10,17
6,4,14
7,13,36
8,5,22


In [ ]:
Comp_colors = ['#FFE500', '#FE8102', '#FF0000', '#07AF00', '#4C7D14', '#414613', '#05C1D9', '#0467FD', '#009588', '#BB2DD4', '#7A00FF', '#4A6876', '#08245B', '#B9461D', '#692108', '#C3C3C3']
neworder = np.array([16,10,7,11,2,12,1,8,4,15,14,5,9,6,3,13]).astype(int) - 1
Comp_colors = np.array(Comp_colors)[neworder]